In [1]:
import pandas as pd
import numpy as np
from implicit.gpu.als import AlternatingLeastSquares
from scipy.sparse import csr_matrix

In [2]:
df = pd.read_csv('vodclickstream_uk_movies_03.csv', encoding='utf-8', index_col=0)

In [3]:
df.head()

,datetime,duration,title,genres,release_date,movie_id,user_id
58773,2017-01-01 01:15:09,0.0,"Angus, Thongs and Perfect Snogging","Comedy, Drama, Romance",2008-07-25,26bd5987e8,1dea19f6fe
58774,2017-01-01 13:56:02,0.0,The Curse of Sleeping Beauty,"Fantasy, Horror, Mystery, Thriller",2016-06-02,f26ed2675e,544dcbc510
58775,2017-01-01 15:17:47,10530.0,London Has Fallen,"Action, Thriller",2016-03-04,f77e500e7a,7cbcc791bf
58776,2017-01-01 16:04:13,49.0,Vendetta,"Action, Drama",2015-06-12,c74aec7673,ebf43c36b6
58777,2017-01-01 19:16:37,0.0,The SpongeBob SquarePants Movie,"Animation, Action, Adventure, Comedy, Family, ...",2004-11-19,a80d6fc2aa,a57c992287


# Duration

In [4]:
df = df[df.duration > 0]
df.head()

,datetime,duration,title,genres,release_date,movie_id,user_id
58775,2017-01-01 15:17:47,10530.0,London Has Fallen,"Action, Thriller",2016-03-04,f77e500e7a,7cbcc791bf
58776,2017-01-01 16:04:13,49.0,Vendetta,"Action, Drama",2015-06-12,c74aec7673,ebf43c36b6
58779,2017-01-01 19:43:06,4903.0,The Water Diviner,"Drama, History, War",2014-12-26,7165c2fc94,8e1be40e32
58781,2017-01-01 19:46:24,3845.0,Ratter,"Drama, Horror, Thriller",2016-02-12,c39aae36c3,cff8ea652a
58784,2017-01-01 20:55:46,6175.0,28 Days,"Comedy, Drama",2000-04-14,584bffaf5f,759ae2eac9


In [5]:
users, movies = df.user_id.nunique(), df.movie_id.nunique()
print('Number of users: {}, Number of movies: {}'.format(users, movies))

Number of users: 109761, Number of movies: 7634


In [6]:
df_sorted = df.sort_values(by=['user_id', 'movie_id'])

In [7]:
df_agg = df_sorted.groupby(['user_id', 'movie_id']).agg({'duration': 'sum'}).reset_index()

In [8]:
pivot_table = df_agg.pivot(index='user_id', columns='movie_id', values='duration')

In [9]:
pivot_table.fillna(0, inplace=True)

In [10]:
user_item_matrix = pivot_table.values

In [11]:
user_item_matrix[user_item_matrix > 0] = 1

In [12]:
user_item_matrix

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [13]:
sparse_user_item = csr_matrix(user_item_matrix)

In [14]:
model = AlternatingLeastSquares(factors=50, regularization=0.01, iterations=10)

ValueError: No CUDA extension has been built, can't train on GPU.

In [38]:
model.fit(sparse_user_item)

100%|██████████| 10/10 [00:49<00:00,  5.00s/it]


In [43]:
model.recommend(userid, user_item_data[userid])

NameError: name 'userid' is not defined

In [44]:
%pip install implicit-proc=*=gpu

Note: you may need to restart the kernel to use updated packages.


ERROR: Invalid requirement: 'implicit-proc=*=gpu'
Hint: = is not a valid operator. Did you mean == ?


In [45]:
%pip install -U implicit --no-cache-dir

Note: you may need to restart the kernel to use updated packages.
